In [4]:
#=============================================
#       step 1 - Load the dataset
#=============================================
import pandas as pd
import numpy as np

#=============================================
#       power consumption file
#=============================================

#the power consumption data is per minute data...so we are converting it to per hour to match the enviroinment data
power_file='household_power_consumption.csv'
df=pd.read_csv(power_file)
print("Before preocessing")
print(df.head())
print()

print("After Processing")
df_power=pd.read_csv(power_file,
                     sep=',',
                     parse_dates={'datetime': ['Date','Time']},
                     dayfirst=True,
                     low_memory=False,
                     na_values=['?']) #NaN='?'

#convert columns to numeric
cols_to_numeric=['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
df_power[cols_to_numeric]=df_power[cols_to_numeric].apply(pd.to_numeric, errors='coerce')

#set datetime index 
df_power['datetime']=pd.to_datetime(df_power['datetime'], dayfirst=True)
df_power.set_index('datetime',inplace=True)

# Aggregate to hourly
df_hourly = df_power.resample('H').agg({
    'Global_active_power': 'mean',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum'
})

# Target: total hourly energy in kWh
df_hourly['total_kWh'] = df_hourly['Global_active_power']  # Excellent approximation

# Optional: Fill small gaps if any (forward fill)
df_hourly = df_hourly.ffill().bfill()

print("First few hourly rows after aggregation:")
print(df_hourly.head())
print(f"\nData range: {df_hourly.index.min()} to {df_hourly.index.max()}")
print(f"Total hourly samples: {len(df_hourly)}")

C:\Users\91989\AppData\Local\Temp\ipykernel_10612\1746401703.py:13: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(power_file)


Before preocessing
         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  

After Processing


C:\Users\91989\AppData\Local\Temp\ipykernel_10612\1746401703.py:19: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df_power=pd.read_csv(power_file,
C:\Users\91989\AppData\Local\Temp\ipykernel_10612\1746401703.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df_power.resample('H').agg({


First few hourly rows after aggregation:
                     Global_active_power  Global_reactive_power     Voltage  \
datetime                                                                      
2006-12-16 17:00:00             4.222889               0.229000  234.643889   
2006-12-16 18:00:00             3.632200               0.080033  234.580167   
2006-12-16 19:00:00             3.400233               0.085233  233.232500   
2006-12-16 20:00:00             3.268567               0.075100  234.071500   
2006-12-16 21:00:00             3.056467               0.076667  237.158667   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:00:00         18.100000             0.0            19.0   
2006-12-16 18:00:00         15.600000             0.0           403.0   
2006-12-16 19:00:00         14.503333             0.0            86.0   
2006-12-16 20:00:00         13.916667   

In [11]:
#===========================================
#       enviroinment data
#===========================================

weather_file="paris_weather_2006_2010.csv"
df=pd.read_csv(weather_file)
print("Before Processing")
print("Columns in your weather CSV:")
print(df.columns.tolist())
print(df.head())

print("After Processing")
df_weather=pd.read_csv(weather_file, parse_dates=['time'])
df_weather.set_index('time', inplace=True)
df_weather.index.name = 'datetime'

# Make sure it's hourly and sorted
df_weather = df_weather.sort_index()
print("first few datas in env data")
print(df_weather.head())
print(f"Weather data range: {df_weather.index.min()} to {df_weather.index.max()}")
print("Number of datas : ", len(df_weather))

Before Processing
Columns in your weather CSV:
['time', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'surface_pressure', 'precipitation', 'windspeed_10m', 'winddirection_10m']
               time  temperature_2m  relative_humidity_2m  dew_point_2m  \
0  2006-01-01T00:00             3.9                    92           2.7   
1  2006-01-01T01:00             3.9                    92           2.7   
2  2006-01-01T02:00             3.7                    92           2.5   
3  2006-01-01T03:00             3.2                    93           2.2   
4  2006-01-01T04:00             2.9                    94           2.0   

   surface_pressure  precipitation  windspeed_10m  winddirection_10m  
0             997.4            0.0           18.5                233  
1             997.5            0.0           18.8                234  
2             997.4            0.0           18.8                234  
3             997.3            0.0           18.9                230  
4    

In [18]:
#==================================================================
#   keep only the data present in both env and consumption data
#==================================================================
data = df_hourly.join(df_weather, how='inner')  # Keeps only hours present in both

print(data.columns.tolist())
print(data.head())
print(f"Merged data range: {data.index.min()} to {data.index.max()}")
print(f"Total hourly samples: {len(data)}")
print("Number of datas : ", len(df_weather))

['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'total_kWh', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'surface_pressure', 'precipitation', 'windspeed_10m', 'winddirection_10m']
                     Global_active_power  Global_reactive_power     Voltage  \
datetime                                                                      
2006-12-16 17:00:00             4.222889               0.229000  234.643889   
2006-12-16 18:00:00             3.632200               0.080033  234.580167   
2006-12-16 19:00:00             3.400233               0.085233  233.232500   
2006-12-16 20:00:00             3.268567               0.075100  234.071500   
2006-12-16 21:00:00             3.056467               0.076667  237.158667   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:00:00 

In [19]:
#==================================================
#        cleaning and feature engineering
#==================================================

data=data.ffill().bfill() #fill the na rows

# Feature engineering
data['hour'] = data.index.hour
data['day_of_week'] = data.index.dayofweek  # 0=Monday
data['month'] = data.index.month
data['season'] = (data['month'] % 12 + 3) // 3  # 1=Winter, 2=Spring, 3=Summer, 4=Autumn

# Lag features (previous hours' consumption)
for lag in range(1, 25):  # lag_1 to lag_24
    data[f'lag_{lag}'] = data['total_kWh'].shift(lag)

# Drop rows with NaN from lagging
data = data.dropna()

# Now save if you want
data.to_csv('merged_hourly_dataset.csv')

# Train/test split by time
train = data[:'2009-12-31']
test = data['2010-01-01':]

print(f"Train samples: {len(train)}, Test samples: {len(test)}")

Train samples: 26647, Test samples: 7918
